In [9]:

import yfinance as yf
import numpy as np
import pandas as pd

from typing import Dict, Any
from scipy.optimize import minimize
from math import exp, log, pi
from scipy.integrate import quad

def get_market_data(
    symbol: str, option_type: str, K: float, expiration: str
) -> pd.DataFrame:
    ticker = yf.Ticker(symbol)
    options_data = ticker.option_chain(expiration)

    if option_type == "put":
        data = options_data.puts
    elif option_type == "call":
        data = options_data.calls
    else:
        raise ValueError("Invalid option type. Choose 'put' or 'call'.")

    # Keep relevant columns and drop missing
    data = data[
        ["strike", "bid", "ask", "volume", "openInterest", "lastPrice"]
    ].dropna()

    # Filter for liquidity
    data = data[
        (data["bid"] > 0)
        & (data["ask"] > 0)
        & (data["openInterest"] > 0)
        & (data["volume"] > 0)
    ]

    # Moneyness filter
    if option_type == "put":
        data = data[data["strike"] < (K * 1.5)]
    else:
        data = data[data["strike"] > (K * 0.5)]

    # Compute mid price
    data["mid_price"] = (data["bid"] + data["ask"]) / 2.0

    # Sort by volume descending and keep top 20
    data = data.sort_values(by="volume", ascending=False).head(20).copy()

    # Compute weights based on volume
    total_volume = data["volume"].sum()
    if total_volume > 0:
        data["volume_weight"] = data["volume"] / total_volume
    else:
        data["volume_weight"] = 1 / len(data)

    return data[["strike", "mid_price", "volume_weight"]].reset_index(drop=True)


In [10]:

symbol = "MSFT"
option_type = "put"
K = 375.00
expiration = "2025-04-04"
S = 378.80
T = 0.02  # Time to expiration in years
r= 0.04
v0= 0.2

data = get_market_data(symbol, option_type, K, expiration)
print(data)

    strike  mid_price  volume_weight
0    375.0      4.125       0.117441
1    380.0      6.200       0.090744
2    370.0      2.675       0.090373
3    385.0      8.975       0.075406
4    372.5      3.325       0.068900
5    350.0      0.435       0.059024
6    377.5      5.075       0.053327
7    367.5      2.115       0.046821
8    360.0      1.045       0.044428
9    320.0      0.095       0.044125
10   345.0      0.310       0.036506
11   355.0      0.660       0.035124
12   365.0      1.685       0.034450
13   315.0      0.090       0.032832
14   342.5      0.255       0.032023
15   325.0      0.115       0.029192
16   390.0     12.550       0.028888
17   382.5      7.500       0.028821
18   387.5     10.675       0.025888
19   330.0      0.150       0.025686


In [13]:
# ---- Heston Characteristic Function ----
# @jit(nopython=True)
def heston_cf(u, S, T, r, v0, kappa, theta, sigma, rho, j):
    x = log(S)
    u_shifted = u - 1j if j == 1 else u
    b = kappa - rho * sigma if j == 1 else kappa
    a = kappa * theta
    d = np.sqrt((rho * sigma * u_shifted * 1j - b)**2 - sigma**2 * (2 * u_shifted * 1j))
    g = (b - rho * sigma * u_shifted * 1j + d) / (b - rho * sigma * u_shifted * 1j - d)

    C = r * u_shifted * 1j * T + (a / sigma**2) * (
        (b - rho * sigma * u_shifted * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T)) / (1 - g))
    )
    D = ((b - rho * sigma * u_shifted * 1j + d) / sigma**2) * (
        (1 - np.exp(d * T)) / (1 - g * np.exp(d * T))
    )
    return np.exp(C + D * v0 + 1j * u_shifted * x)

# ---- Heston Model Price ----
# @jit(nopython=True)
def heston_price(S, K, T, r, v0, kappa, theta, sigma, rho):
    def integrand(u, j):
        phi = heston_cf(u, S, T, r, v0, kappa, theta, sigma, rho, j)
        return np.real(np.exp(-1j * u * np.log(K)) * phi / (1j * u))

    P1 = 0.5 + (1 / pi) * quad(lambda u: integrand(u, 1), 1e-5, 100, limit=200, epsabs=1e-6, epsrel=1e-6)[0]
    P2 = 0.5 + (1 / pi) * quad(lambda u: integrand(u, 2), 1e-5, 100, limit=200, epsabs=1e-6, epsrel=1e-6)[0]
    return S * P1 - K * exp(-r * T) * P2

# ---- Heston Calibration ----
def Calibrate(
    symbol: str,
    option_type: str,
    K: float,
    expiration: str,
    S: float,
    T: float,
    r: float,
    v0: float,
) -> Dict[str, Any]:

    data = get_market_data(symbol, option_type, K, expiration)

    print("data shape:", data.shape)
    
    def objective(params):
        kappa, theta, sigma, rho = params
        error = 0.0
        for _, row in data.iterrows():
            model_price = heston_price(
                S=S, K=row["strike"], T=T, r=r,
                v0=v0, kappa=kappa, theta=theta, sigma=sigma, rho=rho
            )
            diff = model_price - row["mid_price"]
            error += row["volume_weight"] * (diff ** 2)
        return error

    # Initial guess and bounds
    x0 = [1.0, 0.04, 0.5, -0.5]
    bounds = [(0.01, 5), (0.001, 1), (0.01, 2), (-0.99, 0.99)]

    result = minimize(objective, x0, method="L-BFGS-B", bounds=bounds)
    
    print(result)

    if result.success:
        kappa, theta, sigma, rho = result.x
        return {"kappa": kappa, "theta": theta, "sigma": sigma, "rho": rho}
    else:
        raise RuntimeError("Calibration failed.")

In [14]:
result = Calibrate( "MSFT", "put",375.00,"2025-04-04",378.80, 0.02, 0.04, 0.2)
print(result)

data shape: (20, 3)
  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 3935988679.5808697
        x: [ 1.000e-02  1.000e-03  1.878e+00  9.719e-01]
      nit: 15
      jac: [ 2.431e+06 -6.065e+04 -3.039e+05 -5.510e+06]
     nfev: 110
     njev: 22
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
{'kappa': 0.01, 'theta': 0.001, 'sigma': 1.8775329615655567, 'rho': 0.9719460259862395}
